In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import glob

In [ ]:
"""
Importing the required libraries.''
"""
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import numpy as np
import glob

"""
Displaying the sample sketch and color images.
"""
for file in glob.glob('../input/anime-sketch-colorization-pair/data/train/*')[:5]:
    f, a = plt.subplots(1,2, figsize=(10,5))
    a = a.flatten()
    
    img = Image.open(file).convert('RGB')
    print(img)
    a[0].imshow(img.crop((0, 0, 512,512))); a[0].axis('off');
    a[1].imshow(img.crop((512, 0, 1024, 512))); a[1].axis('off');
    
    plt.show()
    print(file)

In [ ]:
'''
"""
Creating a directory for training data. 
"""
get_ipython().system('rm -rf trainData_Image/Imagess')
get_ipython().system('rm -rf trainData_Sketches/Sketches')
get_ipython().system('rm -rf trainData_Images')
get_ipython().system('rm -rf trainData_Sketches')
get_ipython().system('mkdir trainData_Images')
get_ipython().system('mkdir trainData_Sketches')
get_ipython().system('mkdir trainData_Images/Images')
get_ipython().system('mkdir trainData_Sketches/Sketches')
"""
Preprocessing and saving the training data to corresponding directory. 
"""


for idx, file in tqdm(enumerate(glob.glob('../input/anime-sketch-colorization-pair/data/train/*.png'))):
    img = Image.open(file).convert('RGB')
    img.crop((0, 0, 512,512)).save('./trainData_Images/Images/{}.png'.format(idx))
    img.crop((512, 0, 1024, 512)).save('./trainData_Sketches/Sketches/{}.png'.format(idx))


'''



In [ ]:
"""
Creating a directory for validation/test data. 
"""
get_ipython().system('rm -rf testData_Images')
get_ipython().system('rm -rf testData_Sketches')
get_ipython().system('rm -rf testData_Images/Images')
get_ipython().system('rm -rf testData_Sketches/Sketches')
get_ipython().system('mkdir testData_Images')
get_ipython().system('mkdir testData_Sketches')
get_ipython().system('mkdir testData_Images/Images')
get_ipython().system('mkdir testData_Sketches/Sketches')

print('yo')
"""
Preprocessing and saving the validation/test data to corresponding directory. 
"""
for idx, file in tqdm(enumerate(glob.glob('../input/anime-sketch-colorization-pair/data/val/*.png'))):
    if (idx ==1):
        break
    img = Image.open(file).convert('RGB')
    img.crop((0, 0, 512,512)).save('./testData_Images/Images/{}.png'.format(idx))
    img.crop((512, 0, 1024, 512)).save('./testData_Sketches/Sketches/{}.png'.format(idx))

# **import datas**

In [ ]:
! ls testData_Sketches/Sketches

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

i=0
for file in enumerate(glob.glob('./testData_Images/Images/*.png')):
    i+=1
file=glob.glob('./testData_Images/Images/*.png')
op=[]
print(i)  
for i in range (0,1):
    op.append(np.array(Image.open(file[i]).convert('RGB')))
#print(op)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from torch.utils.data import random_split
import glob
##### import CIFAR-10 dataset into trainset, testset# color dataset
transform_color = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.5), (0.5))
    ])

batchSize = 1

testset_color = torchvision.datasets.ImageFolder(root='./testData_Images/',
                                        transform=transform_color)
testloader_color = torch.utils.data.DataLoader(testset_color, batch_size=batchSize,
                                          shuffle=False, num_workers=2)
testset_sketch = torchvision.datasets.ImageFolder(root='./testData_Sketches/',
                                        transform=transform_color)
testloader_sketch = torch.utils.data.DataLoader(testset_sketch, batch_size=batchSize,
                                          shuffle=False, num_workers=2)
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(testloader_color)
images_color, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images_color))

# show dataiter shape
print(images_color.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(testloader_sketch)
images_gray, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images_gray))

# show dataiter shape
print(images_gray.shape)
print("images_gray shape: ", images_gray.shape)

### Define the MLP network

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import glob
import torch.nn as nn
import torch.nn.functional as F

class MLP_net(nn.Module):
    def __init__(self):
        # Input layer is 786432, since gray_image is 3x512x512
        # Output layer is 786432, since color_image is 32x32x3
        super(MLP_net, self).__init__()
        
        self.layer1 = nn.Linear(786432,1024).cuda() #TODO changer ça
        #self.bn1 = nn.BatchNorm1d(1024).cuda()
        self.layer2 = nn.Linear(1024,786432).cuda()
        
    def forward(self, x):
        # convert tensor
        x = x.view(x.size(0), -1)
        
        x = self.layer1(x)
        #x = self.bn1(x)
        x = self.layer2(x)
        return x
print("yo")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
mlp_net = MLP_net().to(device)

print("is cude: ", next(mlp_net.parameters()).is_cuda)


### Define the loss function and optimizer

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(mlp_net.parameters(), lr=0.001, momentum=0.9)

### Tensorboard to save all training output logs

In [ ]:

from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter("./runs_mlp/")

### Train the network

In [ ]:
history_loss=[]
history_val_loss=[]
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    num_training = 0
    for i, data_color in enumerate(testloader_color, 0):
        for j, data_gray in enumerate(testloader_color, 0):
            if (i==j):
                # get the inputs; data is a list of [inputs, labels]
                images_color, labels_color = data_color
                images_gray, labels_color = data_gray 
                # put data in gpu/cpu
                images_color = images_color.to(device)
                images_gray = images_gray.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                # input: images_gray
                images_gray = images_gray.view(images_gray.shape[0],-1)
                print("images_gray: ", images_gray.shape)
                outputs = mlp_net(images_gray)
                # change the shape of images_color: [batchSize,3,32,32] -> [batchSize,3072]
                images_color = images_color.reshape(images_color.shape[0], -1) 
                loss = criterion(outputs, images_color)
                loss.backward()
                optimizer.step()
                # print output statistics
                running_loss += loss.item()
                j = 200             # print every 200 mini-batches
                if i % j == (j-1):     # print every 200 mini-batches
                    print("trainingloss:",'[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / j))
                    writer.add_scalar('Loss/train', running_loss/j, len(trainset_color)*epoch + i)
                    history_loss.append(running_loss / j)
                    running_loss = 0.0
                break
    '''
    running_val_loss = 0.0
    for i, data_val_color in enumerate(valloader_color, 0):
        # get the inputs; data is a list of [inputs, labels]
        images_val_color, labels_val_color = data_val_color
        images_val_gray = rgb2gray_batch(images_val_color)
        # put data in gpu/cpu
        images_val_color = images_val_color.to(device)
        images_val_gray = images_val_gray.to(device)
        
        # forward + backward + optimize
        # input: images_val_gray
        images_val_gray = images_val_gray.view(images_val_gray.shape[0],-1)
        # print("images_val_gray: ", images_val_gray.shape)
        outputs_val = mlp_net(images_val_gray)
        # change the shape of images_val_color: [batchSize,3,32,32] -> [batchSize,3072]
        images_val_color = images_val_color.reshape(images_val_color.shape[0], -1) 
        
        loss_val = criterion(outputs_val, images_val_color)
        
        # print output statistics
        running_val_loss += loss_val.item()
        
        
        j = 10           # print every 200 mini-batches
        if i % j == (j-1):     # print every 200 mini-batches
            print("validation",'[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_val_loss / j))
            writer.add_scalar('Loss/val', running_val_loss/j, len(valset_color)*epoch + i)
            history_val_loss.append(running_val_loss / j)
            
            running_val_loss = 0.0
    '''
writer.flush()
writer.close()
print('Finished Training')

### Loss graphic evolution displayed (on training set and validation set)

In [ ]:
# Defining Figure
f = plt.figure(figsize=(10,7))
f.add_subplot()


#Adding Subplot
plt.plot(range(len(history_loss)), history_loss, label = "loss") # Loss curve for training set
plt.plot(range(len(history_val_loss)), history_val_loss, label = "val_loss") # Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.savefig("Loss_curve.png")
plt.show()

### Inference on train dataset

In [ ]:
# get some random training images
dataiter = iter(trainloader_color)
images_color, labels_color = dataiter.next()
print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [4,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
print(images_color.shape)

images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))

### Inference on validation dataset

In [ ]:
# get some random training images
dataiter = iter(valloader_color)
images_color, labels_color = dataiter.next()
print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [4,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
print(images_color.shape)

images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))

### Inference on test dataset

In [ ]:
# get some random training images
dataiter = iter(testloader_color)
images_color, labels_color = dataiter.next()
print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [batchSize,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
print(images_color.shape)

images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))

images_gray = images_gray.to("cpu")
images_gray = torch.cat((images_gray, images_gray, images_gray), 1)

final_result_display = torch.cat((images_color_show, images_gray, images_color), 0)
imshow(torchvision.utils.make_grid(final_result_display.detach()))